In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from collections import Counter

import os
import re
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import PIL 

import tensorflow as tf
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import glob
from glob import glob

from IPython.display import display
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

%matplotlib inline

In [ ]:
# 1 Download the dataset that is about 350 MB from Kaggle into the local disk and unzip it.
import pathlib
img_dir = 'C:/Users/srgra/OneDrive/Documents/Deep Learning/Homework/Final Data Part 1/indoorCVPR_09/Images'
print(f'The indoorCVPR_09 photes are stored in local directory : {img_dir}')

In [ ]:
total_files = 0
for root, dirs, files in os.walk(str(img_dir)):
    level = root.replace(str(img_dir), '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f'{indent}{os.path.basename(root)}/ ({len(files)} files)')
    total_files += len(files)
print(f'There are {total_files - 1} images in this dataset')

In [ ]:
IndoorImage_dir = [ name for name in list(os.listdir(img_dir)) if os.path.isdir(os.path.join(img_dir, name)) ]
print(f' The Indoor Image labels = {IndoorImage_dir}')

IndoorImage_dir.sort()   
print(f'\n The SORTED Indoor Image labels = {IndoorImage_dir}')

print(f'\nThere are {len(IndoorImage_dir)} classes of Indoor Images.')

In [ ]:
img_paths = glob(os.path.join(img_dir,'*/*.*'))

bad_paths = []

for image_path in img_paths:
    try:
        img_bytes = tf.io.read_file(image_path)
        decoded_img = tf.io.decode_image(img_bytes)
    except tf.errors.InvalidArgumentError as e:
        print(f"Found bad path {image_path}...{e}")
        bad_paths.append(image_path)
        os.remove(image_path)

print("BAD PATHS:")
for bad_path in bad_paths:
    print(f"{bad_path}")

In [ ]:
SEED = 777
os.environ['PYTHONHASHSEED']=str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

for i in range(len(IndoorImage_dir)):
    image_file = glob(os.path.join(img_dir, IndoorImage_dir[i], '*'))
    img = PIL.Image.open(str(image_file[0]))
    
    print(f'(Image size  = ({img.size[0]}, {img.size[1]}, {len(img.mode)}) ; IndoorsPlace = {IndoorImage_dir[i]})')
    display(img)
   

In [ ]:
batch_size = 32
image_height = 256
image_width = 256
split = 0.2

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
  img_dir,
  labels='inferred',
  label_mode='int',
  validation_split= split,
  subset="training",
  seed= 1001,
  image_size=(image_height, image_width),
  batch_size=batch_size)

In [ ]:
val_data = tf.keras.preprocessing.image_dataset_from_directory(
  img_dir,
  labels='inferred',
  label_mode='int',
  validation_split= split,
  subset="validation",
  seed=1001,
  image_size=(image_height, image_width),
  batch_size=batch_size)

In [ ]:
for img, lab in train_data.take(1):
    print(img[1].numpy().astype("uint16"))
    print(f'minimum = {np.amin(img[0].numpy().astype("uint16"))}, maximum = {np.amax(img[0].numpy().astype("uint16"))}')
    break

In [ ]:
plt.figure(figsize=(12, 12))

for img, lab in train_data.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(img[i].numpy().astype("uint16"))
        plt.title(IndoorImage_dir[lab[i]]) 
        plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_data:
    print(f'image_batch.shape = {image_batch.shape} \nlabels_batch.shape = {labels_batch.shape } ')
    break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# 2 Build a baseline CNN model on the training dataset and evaluate it on the test dataset.
model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(2,2),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(67)
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
%%time

callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 3)  
history = model.fit(train_data, validation_data = val_data, epochs = 5, callbacks = [callback], verbose = 1)

In [ ]:
train_history = pd.DataFrame(history.history)
train_history['epoch'] = history.epoch

sns.lineplot(x='epoch', y ='loss', data = train_history)
sns.lineplot(x='epoch', y ='val_loss', data = train_history)
plt.legend(labels=['train_loss', 'val_loss'])

In [ ]:
sns.lineplot(x='epoch', y ='accuracy', data =train_history)
sns.lineplot(x='epoch', y ='val_accuracy', data =train_history)
plt.legend(labels=['train_accuracy', 'val_accuracy'])

In [ ]:
y_pred_prob = model.predict(img)
score = tf.nn.softmax(y_pred_prob)
y_pred = np.argmax(score, axis = 1)
print(classification_report (lab, y_pred))

In [ ]:
# 3 Build a second CNN model with data augmentation and dropout and evaluate it on the test dataset.
data_aug = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", 
    input_shape=(image_height, image_width, 3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.1, width_factor = 0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=(0.1, 0.1))])

In [ ]:
def normalize_image(image, label, target_height = 256, target_width = 256):
    image = tf.cast(image, tf.float32)/255.
    image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)
    return image, label
train_data_normalized = train_data.map(normalize_image, num_parallel_calls = tf.data.experimental.AUTOTUNE)

plt.figure(figsize=(12, 12))
for image, label in train_data_normalized.take(1):
    for i in range(16):
        aug_images = data_aug(image)
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(aug_images[0])
        plt.axis("off")

In [ ]:
model = Sequential([data_aug,
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(2,2),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Dropout(0.25),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dropout(0.25),                  
  layers.Dense(67)
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
%%time

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience= 3)
history = model.fit(train_data, epochs=5, validation_data=(val_data), callbacks=[callback], verbose = 1)

In [ ]:
train_history = pd.DataFrame(history.history)
train_history['epoch'] = history.epoch

sns.lineplot(x='epoch', y ='loss', data = train_history)
sns.lineplot(x='epoch', y ='val_loss', data = train_history)
plt.legend(labels=['train_loss', 'val_loss'])

In [ ]:
sns.lineplot(x='epoch', y ='accuracy', data =train_history)
sns.lineplot(x='epoch', y ='val_accuracy', data =train_history)
plt.legend(labels=['train_accuracy', 'val_accuracy'])

In [ ]:
y_pred_prob = model.predict(img)
score = tf.nn.softmax(y_pred_prob)
y_pred = np.argmax(score, axis = 1)
print(classification_report (lab, y_pred))

In [ ]:
# 4 Build a third CNN model based on the pre-trained model (transfer learning) and evaluate it on the test dataset.
IMG_SHAPE = (image_height, image_width, 3)

MobileNetV3Large_model = tf.keras.applications.MobileNetV3Large(input_shape = IMG_SHAPE, include_top=False, weights='imagenet')

In [ ]:
MobileNetV3Large_model.summary()

In [ ]:
tf.keras.utils.plot_model(MobileNetV3Large_model, show_shapes=True)

In [ ]:
MobileNetV3Large_model.trainable = False

preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input

In [ ]:
image_batch, label_batch = next(iter(train_data))
feature_batch = MobileNetV3Large_model(image_batch)
print(feature_batch.shape)

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
Flatten_layer = tf.keras.layers.Flatten()
feature_batch_average = Flatten_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(67)
prediction_batch = prediction_layer(feature_batch_average)
print(f' The size of the predicted value for a given batch = {prediction_batch.shape}')
print(prediction_batch)

In [ ]:
inputs = tf.keras.Input(shape = IMG_SHAPE)

x = data_aug(inputs)
x = preprocess_input(x)

x = MobileNetV3Large_model(x, training=False)

x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)

outputs = prediction_layer(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
%%time

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience= 3)
history = model.fit(train_data, epochs=5, validation_data=(val_data), callbacks=[callback], verbose = 1)

In [ ]:
train_history = pd.DataFrame(history.history)
train_history['epoch'] = history.epoch

sns.lineplot(x='epoch', y ='loss', data = train_history)
sns.lineplot(x='epoch', y ='val_loss', data = train_history)
plt.legend(labels=['train_loss', 'val_loss'])

In [ ]:
sns.lineplot(x='epoch', y ='accuracy', data =train_history)
sns.lineplot(x='epoch', y ='val_accuracy', data =train_history)
plt.legend(labels=['train_accuracy', 'val_accuracy'])

In [ ]:
y_pred_prob = model.predict(img)
score = tf.nn.softmax(y_pred_prob)
y_pred = np.argmax(score, axis = 1)
print(classification_report (lab, y_pred))

In [ ]:
# 5 Which model do you recommend for the model in Q2, Q3, and Q4? Justify your answer.
# I would recommend the model from Q2, as the accuracy was 0.94. The accuracy of Q3 was only 0.16 and the accuracy of Q4 was
# only 0.59.